In [1]:
!pip install elasticsearch sentence-transformers pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
es = Elasticsearch(
    hosts=["http://localhost:9200"],
    # Nếu không cần auth thì không cần user/pass
)
# Test kết nối
assert es.ping(), "Elasticsearch không kết nối được!"


AssertionError: Elasticsearch không kết nối được!

In [ ]:
index_name = "places_danang"

mapping = {
    "mappings": {
        "properties": {
            "type": {"type": "keyword"},
            "name": {"type": "text"},
            "description": {"type": "text"},
            "time": {"type": "keyword"},
            "price": {"type": "keyword"},
            "location": {"type": "text"},
            "area": {"type": "keyword"},
            "note": {"type": "text"},
            "id": {"type": "keyword"},
            "full_text": {"type": "text"},  # Dùng cho BM25
            "vector_search": {
                "type": "dense_vector",
                "dims": 384,  # all-MiniLM-L6-v2 output 384 dimensions
                "index": True,           # Bật index cho vector search (Elastic >=8.5)
                "similarity": "cosine"   # hoặc "l2_norm"
            }
        }
    }
}


In [ ]:
#Delete pervious index and create a new one: 
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=mapping)
print(f"Index `{index_name}` đã được tạo!")


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
df = pd.read_csv("data_danang_ok.csv")  # Đường dẫn file của bạn

# Tạo embedding cho từng record
def embed(text):
    return model.encode(text).tolist()

# Nếu cột vector_search đã có, bỏ qua đoạn này, còn không:
tqdm.pandas()
df["vector_search"] = df["full_text"].progress_apply(embed)

#indexing data to elasticsearch
for i, row in tqdm(df.iterrows(), total=len(df)):
    doc = row.to_dict()
    # Nếu vector_search dạng numpy, cần chuyển sang list
    es.index(index=index_name, id=doc["id"], document=doc)